# 第1章 はじめに

## 1.1 transformersを使って自然言語処理を解いてみよう
大規模言語モデルをプログラムから扱うための標準的なライブラリが**transformers**
<br>
Pythonのパッケージマネージャであるpipコマンドを使用してtransformersをインストールする。

In [ ]:
!pip install transformers[ja,sentencepiece,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 85.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.9/697.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.5 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=4d128f5499186e2d377ff0be77260a44a882b11fe26cca36f2db2f4faf0a464c
  Stored in directory: /root/.cache/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7403 sha256=c64dd0e696d5197ef30694eb5f7f35300f40e1fade6959f6168bbf9f

In [ ]:
from transformers import pipeline

### 1.1.1 文書分類
ここでは**感情分析**を行う。感情分析モデルlim-book/bert-base-japanese-v3-marc.jaは通販サイトのレビュー記事で訓練されており、テキストがpositiveであるかnegativeであるかを予測できる。

In [ ]:
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する音楽がある。"
# positive_textの極性を予測
print(text_classification_pipeline(positive_text)[0])

In [ ]:
# negative_textの極性を予測
negative_text = "世界には言葉がでないほどひどい音楽がある。"
print(text_classification_pipeline(negative_text)[0])

"score"は予測確率を示したものなので、上記の例からいずれも96%の高い確率で妥当なラベルを予測している。

### 1.1.2 自然言語推論
**自然言語推論**は、2つのテキストの論理関係を予測するタスクであり、言語モデルの意味理解能力を評価するために使用される。

In [ ]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"

# textとentailment_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": entailment_text}))

"entailment"とは含意を意味している。

In [ ]:
contradiction_text = "二人の男性が飛んでいます"
# textとcontradiction_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

"contradiction"は矛盾を意味している。

In [ ]:
neutral_text = "2人の男性が、白い飛行機を眺めています"
# textとneutral_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": neutral_text}))

"neutral"は中立であり、含意とも矛盾とも取れないときに出力される。

### 1.1.3 意味的類似度計算
**意味的類似度計算**は2つのテキストの意味が似ている度合いをスコアとして予測するタスクである。**情報検索**や**複数テキストの内容の整合性**を確認する際に役に立つ。このモデルは与えられた2つのテキストの意味的類似度を0から5の範囲で予測する。

In [ ]:
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none",
)
text = "川べりでサーフボードを持った人たちがいます"
sim_text = "サーファーたちが川べりに立っています"
# textとsim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

In [ ]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
# textとdissim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

また**chapter08**ではテキストの意味をベクトルで表現する**文埋め込み**モデルを紹介する。このモデルから得られるテキストのベクトルのコサイン類似度を意味的類似度とみなすことができる。コサイン類似度の値の範囲は-1から1であることに注意して、以下のコードを実行してみる。

In [ ]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

# textとsim_textのベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
# textとsim_textの類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

In [ ]:
# dissim_textのベクトルを獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
# textとdissim_textの類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

### 1.1.4 固有表現認識
**固有表現認識**とはテキストに含まれる固有表現を抽出するタスクである。**テキストデータから必要な情報を抽出するための基本的なタスクの一つである**。
<br>
試しに「大谷翔平は岩手県水沢市出身のプロ野球選手」という文から固有表現を抽出してみる。

In [ ]:
from pprint import pprint

ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)
text = "大谷翔平は岩手県水沢市出身のプロ野球選手"
# text中の固有表現を抽出
pprint(ner_pipeline(text))

### 1.1.5 要約生成
**要約生成**は比較的長い文章から短い要約を生成するタスクである。

In [ ]:
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)
article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"
# articleの要約を生成
print(text2text_pipeline(article)[0]["generated_text"])

自然言語処理にはこれ以外に多くの機能を兼ね備えている。
- 機械翻訳 : ある言語で記述されたテキストを別の言語に翻訳するタスク
- 対話システム : コンピュータが人間と対話するタスク
- 形態素解析 : 意味を持つ最小の単位である**形態素**に文を分割して解析する処理
- 共参照解析 : 異なる名詞が同一のものを指しているかどうかを識別する処理

## 1.2 transformersの基本的な使い方
基本的に**Auto Classes**というクラス群を用いる。適切な実装を自動的に選択してくれる。
<br>
大規模言語モデルを含むおおくの自然言語処理のモデルでは、テキストを細かい単位に分割してからモデルに入力する。モデルが扱う基本的な単位を**トークン**、トークンに分割する処理を**トークナイゼーション**、トークン単位に分割する実装を**トークナイザ**と呼ぶ。以下のコードはAutoTokenizerを使ってテキストをトークンに分割する処理である。

In [ ]:
from transformers import AutoTokenizer

# AutoTokenizerでトークナイザをロードする
tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")
# 入力文をトークンに分割する
tokenizer.tokenize("今日は天気が良いので")

In [ ]:
from transformers import AutoModelForCausalLM

# 生成を行うモデルであるAutoModelForCausalLMを使ってモデルをロードする
model = AutoModelForCausalLM.from_pretrained(
    "abeja/gpt2-large-japanese"
)
# トークナイザを使ってモデルへの入力を作成する
inputs = tokenizer("今日は天気が良いので", return_tensors="pt")
# 後続のテキストを予測
outputs = model.generate(
    **inputs,
    max_length=15,  # 生成する最大トークン数を15に指定
    pad_token_id=tokenizer.pad_token_id  # パディングのトークンIDを指定
)
# generate関数の出力をテキストに変換する
generated_text = tokenizer.decode(
    outputs[0], skip_special_tokens=True
)
print(generated_text)

## 1.3 単語埋め込みとニューラルネットワークの基礎
単語の意味をコンピュータで扱う方法である**単語埋め込み**についてみていく。
<br>
まず、単語の意味をコンピュータに教えるにはどうしたらよいか。従来の自然言語処理では**WordNet**に代表されるような人手で佐生精された単語同士の関係を記述した辞書を使って、コンピュータで意味を表現する方法が研究されてきた。
<br>
こうしたなかで、単語の意味を表現したベクトルを大規模なテキストから学習できることを示したニューラルネットワークが2013年に発表された。**単語ベクトル**とよび、自然言語処理に用いるために構築されたテキストを**コーパス**と呼ぶ。これは、ある単語の意味は周辺に出現する単語によって表せると考える**分布仮設**に基づいて設計されている。
<br>
例としてskip-gramを用いて「今日こたつでみかんを食べる」という文から単語の意味を学習することを考える。skip-gramでは、中央の単語から左側と右側の窓幅文の周辺単語を予測することで学習が行われる。単語の予測確率は**softmax function**を用いて計算される。出力は要素の合計が1であるため確率分布ととらえられる。
<br>
$$
softmax_m(c)=\frac{\exp(c_m)}{\Sigma_{k=1}^{K}\exp(c_k)}
$$
<br>
損失関数の最小化は**勾配法**を用いて行うことができる(勾配降下法)。
<br>
$$
\theta^{(t+1)}=\theta^{(t)}-\alpha \nabla_{\theta}L(\theta)
$$
<br>
ここで$\alpha$は学習率とされ、一度にどの程度の大きさでパラメータを更新するかを制御している。ステップごとに訓練コーパス中の全単語について求めるのは計算負荷が大きいので、**ミニパッチ**をしようして、**確率的勾配降下法**により求める。
<br>
また、勾配を求める際には**誤差逆伝搬法**が標準的に用いられる。この方法は損失を**forward computation**で計算した後に、微分の連鎖律を利用して損失を逆方向に伝搬させることで各パラメータの勾配を計算する方法である。
<br>
word2vecのように、モデルをあらかじめ別のタスクで訓練することを**事前学習**、事前学習したモデルを適用する先のタスクのことを**下流タスク**と呼ぶ。ある解きたいタスクに対して別の方法で学習したモデルを転用する方式を**転移学習**という。入力から自動的に予測するラベルを生成して学習を行う方式を**自己教師あり学習**という。

## 1.4 大規模言語モデルとは
word2vecの登場以降、文脈を考慮した単語埋め込みである**文脈化単語埋め込み**を大規模なコーパスから自己教師あり学習で獲得するモデルが提案された。これとほぼ同時期に、機械翻訳のモデルとして後述する**Transformer**という優れたニューラルネットワークが提案された。